In [1]:
import numpy as np
import pandas as pd
import scipy
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
sns.set_palette('muted')

## Exogenous Variables
- Etherem: OHLCV
- Other Cryptocurrency prices
    - BTC: Bitcoin
    - XRP: Ripple
    - EOS: EOS.IO
    - LTC: Litecoin
    - XLM: Stellar
    - XMR: Monero
- Other Indices
    - VIX: CBOE Volatility Index
    - TWEXB: Trade Weighted USD Index, Broad
    - EFFR: Effective Federal Fund Rate

### Load Data

In [2]:
date = dt.datetime.today().strftime('%Y-%m-%d')
symbols = ['eth','btc','xrp','eos','ltc','xlm','xmr','vixcls','twexb','effr']

In [3]:
# create empty df for each symbol got data for
d = {symbol: pd.DataFrame() for symbol in symbols}

# populate each with their respective json data
for symbol, df in d.items():
    d[symbol] = pd.read_json('{}_data_{}.json'.format(symbol, date), orient='split')

# convert keys:values to variables = assignments
locals().update(d)

### DataFrame Cleaning and Prep

In [4]:
# rename target currency columns with currency name
cols_orig = list(eth.columns)
cols_new = ['date']

for col in cols_orig[1:]:
    cols_new.append('eth_{}'.format(col))
    
eth.columns = cols_new

In [5]:
# prep exogenous cryptocurrencies
exog_curr = [btc, xrp, eos, ltc, xlm, xmr]

for curr, symbol in zip(exog_curr, symbols[1:-3]):
    curr.drop(['open', 'high', 'low', 'volumefrom', 'volumeto'],
             axis=1,
             inplace=True)
    curr.rename({'close':symbol}, axis=1, inplace=True)

In [6]:
# do this the dumb way for now
df = eth.merge(btc, how='outer', on='date')
df = df.merge(xrp, how='outer', on='date')
df = df.merge(eos, how='outer', on='date')
df = df.merge(ltc, how='outer', on='date')
df = df.merge(xlm, how='outer', on='date')
df = df.merge(xmr, how='outer', on='date')
df = df.merge(vixcls, how='outer', on='date')
df = df.merge(twexb, how='outer', on='date')
df = df.merge(effr, how='outer', on='date')

In [7]:
# start at beginning of oldest cryptocurrency
currencies = exog_curr
currencies.append(eth)
start_date = pd.to_datetime(
    dt.datetime.today().strftime('%Y-%m-%d'),
    format='%Y-%m-%d')

for curr in exog_curr:
    if curr.date.min() < start_date:
        start_date = curr.date.min()

print('earliest date: {}'.format(start_date))

earliest date: 2010-07-16 00:00:00


In [8]:
df.sort_values(by='date', inplace=True)
df = df.loc[df.date >= start_date]
df.reset_index(drop=True, inplace=True)

# replace '.' with nan so ffill will catch everything
# '.' denotes a holiday
df.replace('.', np.nan, inplace=True)
df.ffill(inplace=True)

# backwards fill index without daily frequency
df.twexb.bfill(inplace=True)
df.head(15)

,date,eth_open,eth_high,eth_low,eth_close,eth_volumefrom,eth_volumeto,btc,xrp,eos,ltc,xlm,xmr,vixcls,twexb,effr
0,2010-07-16,NaN,NaN,NaN,NaN,NaN,NaN,0.04951,NaN,NaN,NaN,NaN,NaN,26.25,103.1938,0.19
1,2010-07-17,NaN,NaN,NaN,NaN,NaN,NaN,0.08584,NaN,NaN,NaN,NaN,NaN,26.25,103.1938,0.19
2,2010-07-18,NaN,NaN,NaN,NaN,NaN,NaN,0.08080,NaN,NaN,NaN,NaN,NaN,26.25,103.1938,0.19
3,2010-07-19,NaN,NaN,NaN,NaN,NaN,NaN,0.07474,NaN,NaN,NaN,NaN,NaN,25.97,103.1938,0.19
4,2010-07-20,NaN,NaN,NaN,NaN,NaN,NaN,0.07921,NaN,NaN,NaN,NaN,NaN,23.93,103.1938,0.18
5,2010-07-21,NaN,NaN,NaN,NaN,NaN,NaN,0.05050,NaN,NaN,NaN,NaN,NaN,25.64,103.1938,0.18
6,2010-07-22,NaN,NaN,NaN,NaN,NaN,NaN,0.06262,NaN,NaN,NaN,NaN,NaN,24.63,103.1938,0.18
7,2010-07-23,NaN,NaN,NaN,NaN,NaN,NaN,0.05454,NaN,NaN,NaN,NaN,NaN,23.47,103.1938,0.19
8,2010-07-24,NaN,NaN,NaN,NaN,NaN,NaN,0.05050,NaN,NaN,NaN,NaN,NaN,23.47,103.1938,0.19
9,2010-07-25,NaN,NaN,NaN,NaN,NaN,NaN,0.05600,NaN,NaN,NaN,NaN,NaN,23.47,103.1938,0.19


In [10]:
df.to_json('df.json', orient='split')

### get all this to work later

In [ ]:
for symbol, df in d.items():
    print(df.date.min())

In [ ]:
# start at beginning of oldest cryptocurrency
currencies = exog_curr
currencies.append(eth)
start_date = pd.to_datetime(
    dt.datetime.today().strftime('%Y-%m-%d'),
    format='%Y-%m-%d')

for curr in exog_curr:
    if curr.date.min() < start_date:
        start_date = curr.date.min()

print('earliest date: {}'.format(start_date))

for symbol, df in d.items():
    if df.date.min() < start_date:
        print('\n{} original min date: {}'.format(symbol, df.date.min()))
        #df.loc[df.date >= start_date
        print('{} updated min date: {}'.format(symbol, df.date.min()))

In [ ]:
#df.set_index('date', inplace=True)
#df[df.date >= '2010-07-22']
df.head()

In [ ]:
df.drop(df[x <= '2010-07-22'], inplace=True)

In [ ]:
# address missinginess
for symbol, df in d.items():
    print(df.isnull().any().sum())

In [ ]:
vixcls.date.min()

In [ ]:
print(eth.date[0])

### Create Single DataFrame

In [ ]:
df = pd.DataFrame()

### **[VIX: CBOE Volatility Index](https://en.wikipedia.org/wiki/VIX)**
- Measure of stock market's expectation of volatility implied by S&P 500 index options, aka 'fear index'
- [Data Source](http://www.cboe.com/products/vix-index-volatility/vix-options-and-futures/vix-index/vix-historical-data)

In [ ]:
df = eth.copy()

In [ ]:
vix_raw = pd.read_csv('vixcurrent.csv', header=1)
print(vix_raw.info())
vix_raw[:1]

In [ ]:
start_date = '2015-08-06'
end_date = '2018-09-09'
vix_close = vix_raw[['Date', 'VIX Close']].copy()
vix_close['Date'] = pd.to_datetime(vix_close['Date'], format='%m/%d/%Y')
vix_close = vix_close.loc[(vix_close['Date'] >= start_date)
                          & (vix_close['Date'] <= end_date)]
vix_close.sort_values('Date', inplace=True)
vix_close.reset_index(drop=True, inplace=True)
vix_close.columns = ['date', 'vix']
vix_close.head()

In [ ]:
df = df.merge(vix_close, how='outer', on='date')
df.vix.ffill(inplace=True)
#df_04ex.vix[0]= vix_raw.loc[vix_raw['Date'] == '11/24/2017', ['VIX Close']].values
print(df.vix.isnull().sum())
df.head()

### **[DXY: US Dollar Index](https://en.wikipedia.org/wiki/U.S._Dollar_Index)**
- Measure of value of USD relative to a basket of foreign currencies
- [Data Source](https://quotes.wsj.com/index/DXY/historical-prices)

In [ ]:
dxy_raw = pd.read_csv('dxy_historical.csv')
print(dxy_raw.info())
dxy_raw[:1]

In [ ]:
dxy = dxy_raw[['Date', ' Close']].copy()
dxy['Date'] = pd.to_datetime(dxy['Date'], format='%x')
dxy = dxy.loc[(dxy['Date'] >= start_date)
              & (dxy['Date'] <= end_date)]
dxy.sort_values('Date', inplace=True)
dxy.reset_index(drop=True, inplace=True)
dxy.columns = ['date', 'dxy']
dxy.head()

In [ ]:
df = df.merge(dxy, how='outer', on='date')
df.ffill(inplace=True)

# get first value
#dxy_raw['Date'] = pd.to_datetime(dxy_raw['Date'], format='%x')
#df_04ex.dxy[0] = dxy_raw.loc[dxy_raw['Date'] == '11/24/2017', [' Close']].values
print(df.dxy.isnull().sum())
df.head(2)

In [ ]:
df.tail(2)

### **[EFFR: Federal Funds Rate](https://en.wikipedia.org/wiki/Federal_funds_rate)**
- The interest rate at which depository institutions (banks and credit unions) lend reserve balances to other depository institutions overnight, on an uncollateralized basis 
- [Data Source](https://fred.stlouisfed.org/series/EFFR)

In [ ]:
effr_raw = pd.read_csv('EFFR.csv')
effr_raw.head()

In [ ]:
effr = effr_raw.copy()
effr.columns = ['date', 'effr']
effr['date'] = pd.to_datetime(effr['date'], format='%Y-%m-%d')
effr.sort_values('date', inplace=True)
effr = effr.loc[(effr['date'] >= start_date) & (effr['date'] <= end_date)]
effr.reset_index(drop=True, inplace=True)
effr.head()

In [ ]:
df = df.merge(effr, how='outer', on='date')
df.ffill(inplace=True)
print(df.effr.isnull().sum())
df.head()

## Add other coins

In [ ]:
# bitcoin
btc_raw = pd.read_csv('btc_raw.csv')
btc = btc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
btc['timestamp'] = pd.to_datetime(btc['timestamp'])
btc['timestamp'] = btc['timestamp'].dt.date
btc.rename(columns = {'timestamp':'date', 'close':'btc'}, inplace=True)
btc['date'] = pd.to_datetime(btc['date'], format='%Y-%m-%d')
btc = btc.loc[(btc['date'] >= start_date) & (btc['date'] <= end_date)]
btc.sort_values(by='date', inplace=True)
btc.reset_index(drop=True, inplace=True)
btc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
btc.head(2)

In [ ]:
df = df.merge(btc, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# ripple
xrp_raw = pd.read_csv('xrp_raw.csv')
xrp = xrp_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xrp['timestamp'] = pd.to_datetime(xrp['timestamp'])
xrp['timestamp'] = xrp['timestamp'].dt.date
xrp.rename(columns = {'timestamp':'date', 'close':'xrp'}, inplace=True)
xrp['date'] = pd.to_datetime(xrp['date'], format='%Y-%m-%d')
xrp = xrp.loc[(xrp['date'] >= start_date) & (xrp['date'] <= end_date)]
xrp.sort_values(by='date', inplace=True)
xrp.reset_index(drop=True, inplace=True)
xrp.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xrp, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# eos.io
eos_raw = pd.read_csv('eos_raw.csv')
eos = eos_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
eos['timestamp'] = pd.to_datetime(eos['timestamp'])
eos['timestamp'] = eos['timestamp'].dt.date
eos.rename(columns = {'timestamp':'date', 'close':'eos'}, inplace=True)
eos['date'] = pd.to_datetime(eos['date'], format='%Y-%m-%d')
eos = eos.loc[(eos['date'] >= start_date) & (eos['date'] <= end_date)]
eos.sort_values(by='date', inplace=True)
eos.reset_index(drop=True, inplace=True)
eos.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(eos, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# litecoin
ltc_raw = pd.read_csv('ltc_raw.csv')
ltc = ltc_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
ltc['timestamp'] = pd.to_datetime(ltc['timestamp'])
ltc['timestamp'] = ltc['timestamp'].dt.date
ltc.rename(columns = {'timestamp':'date', 'close':'ltc'}, inplace=True)
ltc['date'] = pd.to_datetime(ltc['date'], format='%Y-%m-%d')
ltc = ltc.loc[(ltc['date'] >= start_date) & (ltc['date'] <= end_date)]
ltc.sort_values(by='date', inplace=True)
ltc.reset_index(drop=True, inplace=True)
ltc.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(ltc, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# stellar
xlm_raw = pd.read_csv('xlm_raw.csv')
xlm = xlm_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xlm['timestamp'] = pd.to_datetime(xlm['timestamp'])
xlm['timestamp'] = xlm['timestamp'].dt.date
xlm.rename(columns = {'timestamp':'date', 'close':'xlm'}, inplace=True)
xlm['date'] = pd.to_datetime(xlm['date'], format='%Y-%m-%d')
xlm = xlm.loc[(xlm['date'] >= start_date) & (xlm['date'] <= end_date)]
xlm.sort_values(by='date', inplace=True)
xlm.reset_index(drop=True, inplace=True)
xlm.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xlm, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
# monero
xmr_raw = pd.read_csv('xmr_raw.csv')
xmr = xmr_raw[['timestamp', 'open', 'high', 'low', 'close', 'volumefrom', 'volumeto']].copy()
xmr['timestamp'] = pd.to_datetime(xmr['timestamp'])
xmr['timestamp'] = xmr['timestamp'].dt.date
xmr.rename(columns = {'timestamp':'date', 'close':'xmr'}, inplace=True)
xmr['date'] = pd.to_datetime(xmr['date'], format='%Y-%m-%d')
xmr = xmr.loc[(xmr['date'] >= start_date) & (xmr['date'] <= end_date)]
xmr.sort_values(by='date', inplace=True)
xmr.reset_index(drop=True, inplace=True)
xmr.drop(['high', 'low', 'open', 'volumefrom', 'volumeto'], axis=1, inplace=True)
df = df.merge(xmr, how='outer', on='date')
print(df.shape)
df.head(3)

In [ ]:
df.to_pickle('crypto_df.pkl')

In [ ]:
df.tail()

In [ ]:
print(df.xlm.isnull().sum())